In [1]:
#import streamlit as st
import pandas as pd

In [2]:
from Features import world_constructions as WCf

In [3]:
from Features import search_from_luxury_brands as Sf

### Loading Postgress

In [4]:
df = WCf.world_construction.init_luxury_gallery()
df.head(2)

,id,prod_id,Detail,Summary,Brand,Price,base64
0,161,1,Type: Shirt / Fit: Regular fit / ...,The item is a Gucci silk shirt featuring an op...,Gucci,1500.0,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
1,162,2,Type: Shirt / Fit: Regular fit / Sleeve...,The Gucci cotton Oxford GG shirt with an optic...,Gucci,650.0,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."


In [5]:
description = df.iloc[0].Summary

In [6]:
description

'The item is a Gucci silk shirt featuring an optical GG monogram print. It is designed with a regular fit and features long sleeves with buttoned cuffs. The neckline is a classic pointed collar, adding a touch of formality. The material used is high-quality silk, which gives the shirt a luxurious feel and fluid drape, suitable for all seasons due to its breathability and lightweight nature. The primary colors used are a deep lavender (5467) and white, which are intricately interwoven to create a striking visual effect. The pattern placement is consistent throughout the shirt, ensuring a cohesive look. The texture is smooth and slightly glossy, typical of silk fabric, with no transparency, making it appropriate for both formal and semi-formal occasions. There are no pockets, embellishments, or lapels, which maintains a sleek and streamlined silhouette. The shirt’s length is standard, typically falling just below the waistline, and it is finished with concealed buttons for a minimalist a

In [7]:
df_retail = WCf.world_construction.init_retail_gallery()
df_retail.head()

,id,Brand_id,Detail,Summary,Brand,Price
0,174,14,Type: Shirt / Fit: Regular fit / Sleeve...,"The shirt in the images is a contemporary, cas...",HM,9.99
1,175,15,Type: Shirt / Fit: Regular fit / Sleeve...,"The shirt featured in the images is a casual, ...",HM,9.99
2,176,16,Type: Shirt / Fit: Regular fit / Sleeve...,The shirt depicted in the images is a casual y...,HM,9.99
3,177,17,"Type: Top, Blouse / Fit: Regular fit / Slee...",The garment depicted in the images is a women'...,HM,9.99
4,178,18,Type: Long-sleeve top / Fit: Regular fit / ...,The garment is a casual women's long-sleeve sh...,HM,9.99


In [8]:
df_retail

### Loading Chroma

In [8]:
vectorestore = WCf.world_construction.init_chroma_db()

c:\Users\juand\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\juand\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\juand\anaconda3\Lib\site-packages\transformers\utils\generic.py:482: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\

In [9]:
query = "Summer"
docs = vectorestore.similarity_search(query)

In [10]:
docs[0].page_content

'The garment in the images is a crochet top made from a cotton-blend fabric, showcasing a relaxed and breathable fit suitable for warm seasons such as spring and summer. It features a sleeveless design with wide straps and a round neckline, providing a casual and comfortable aesthetic. The top has a distinct crochet texture that adds a touch of bohemian charm and handcrafted appeal. The material appears to have medium weight, offering moderate fluidity and drape, enhancing its laid-back, breezy style. It comes in a solid, warm beige color, which is versatile and neutral, making it easy to pair with various outfits. The transparency of the crochet pattern adds a subtle, airy feel, perfect for hot weather while still maintaining coverage. There are no visible pockets, lapels, or embellishments, keeping the design simple and unembellished. This top is ideal for casual, daytime occasions, such as beach outings, picnics, or casual meet-ups.'

## Feature 1: Find similar pieces from luxury brands in retail brands 

In [11]:
entity = Sf.seacrh_from_luxury_brands(description=description, vectorstore=vectorestore)

In [12]:
answer = entity.search_similarity_from_description()

### Organize in daframe for display

In [15]:
answer['answer']

"I'm sorry, I couldn't find a match for the specific silk shirt with an optical monogram print as described."

In [16]:
answer['context'][0].page_content

"This shirt is a straight-fit, long-sleeve top made from 100% linen, ensuring breathability and comfort, making it ideal for warmer seasons like spring and summer. The shirt features a classic shirt collar and a button-down front, contributing to its versatile and semi-formal style. The neckline is a standard collar style, and the sleeves are long with buttoned cuffs, allowing for a tailored look. The material is lightweight and has a natural, slightly textured finish typical of linen, enhancing its casual yet polished appearance. The shirt is primarily a soft, muted beige (often referred to as ecru), which gives it a neutral and sophisticated look, suitable for various occasions including casual outings and semi-formal events. There are no visible pockets, embellishments, or patterns, emphasizing a clean and minimalist design. The fabric's fluidity is moderate, providing structure while still allowing ease of movement. Overall, this shirt is a versatile and stylish piece that can be d

In [17]:
answer['context'][0].metadata['doc_id']

'38'

In [18]:
len(answer['context'])

5

In [19]:
def organize_data(answer):

    page_content = []
    metadata_1 = []
    metadata_2 = []
    doc_id =[]


    for i in range(0,4):

        doc_id.append(answer['context'][i].metadata['doc_id'])
        page_content.append(answer['context'][i].page_content)
        metadata_1.append(answer['context'][i].metadata['img_1'])
        metadata_2.append(answer['context'][i].metadata['img_2'])

    df_init = pd.DataFrame([doc_id,page_content,metadata_1,metadata_2]).T

    df_init = pd.DataFrame(df_init)

    df_init.rename(columns={0:"Id",1:"Summary",2:"Img_1",3:"Img_2"},inplace=True)


    return df_init

    

In [20]:
df_recommendations = organize_data(answer=answer)
df_recommendations

,Id,Summary,Img_1,Img_2
0,38,"This shirt is a straight-fit, long-sleeve top ...","data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ...","data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
1,36,The item is a women's casual shirt with a rela...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ...","data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
2,20,"The shirt depicted in the images is a casual, ...","data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ...","data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
3,27,"The shirt depicted in the images is a casual, ...","data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ...","data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."


In [21]:
for index, row in df_recommendations.iterrows():
    print(row['Id'])

38
36
20
27


## Feature 2: Find pieces from user prompt description ( Text prompt to suggestions)

### Feature 3: Find pieces similar to user provided piece ( Image to suggestions)